# Построение системы, способной проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра» 

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами.В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. 
-  Поcтроим модель для задачи классификации, которая выберет подходящий тариф. 
-  Построим модель с максимально большим значением accuracy. 
-  Доведем долю правильных ответов до 0.75. Проверим accuracy на тестовой выборке самостоятельно.

## Изучение данных из файла

Откроем файл с данными и изучим общую информацию.

Путь к файлу:

-  datasets/users_behavior.csv

Описание данных:

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. 

Известно:

- **сalls** — количество звонков,
- **minutes** — суммарная длительность звонков в минутах,
- **messages** — количество sms-сообщений,
- **mb_used** — израсходованный интернет-трафик в Мб,
- **is_ultra** — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).


Таким образом, исследование пройдёт в **пять этапов**:

- Изучим данные и разделим исходные данные на обучающую, валидационную и тестовую выборки.
- Исследуем качество разных моделей, меняя гиперпараметры. Кратко напишите выводы исследования.
- Проверим качество модели на тестовой выборке.
- Попробуем проверить модели на вменяемость.
- Общий вывод.

Исследование:

In [1]:
# импорт библиотек

import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


# импорт функции расчёта accuracy из библиотеки sklearn 

from sklearn.metrics import accuracy_score

from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC

In [2]:
# чтение файла с данными и сохранение  
df = pd.read_csv('/datasets/users_behavior.csv') 

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


**Вывод**

- Датасет состоит из 3214 объектов и 5 признаков, 4 столбца float64, 1 столбец int64.
- Признак, который нужно предсказать, — целевой: в нашей задаче это is_ultra, модель в нашем проекте будет предсказывать тариф «Ультра» — 1, тариф «Смарт» — 0. Наш целевой признак является категориальным.

## Разделим исходные данные на обучающую, валидационную и тестовую выборки

Разделите данные на обучающую и валидационную выборки:
- Функция train_test_split() возвращает два новых набора данных — обучающий и валидационный

In [6]:
df_train, df_valid = train_test_split(df, test_size=0.4, random_state=12345) 

0.2 разделим на валидационную, 0.2 – на тестовую:
- Функция train_test_split() возвращает два новых набора данных — валидационный и тестовый

In [7]:
df_valid, df_test = train_test_split(df_valid, test_size=0.5, random_state=12345)

**Вывод:**

- Обучающая модель  df_train - 60 %
- Валидационная модель df_valid - 20 %
- Тетовая модель df_test - 20 %

# Исследуем качество разных моделей, меняя гиперпараметры

Объявим две переменные:
- features (англ. «признаки») — запишем в неё признаки;
- target (англ. «цель») — целевой признак.

In [8]:
# создадим переменные для признаков и целевого признака и запишем в них код
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

## Решающее дерево

Рассмотрим одну популярную модель — **решающее дерево**. Оно может описывать процесс принятия решения почти в любой ситуации.

In [9]:

for depth in range(1, 10):
    # обучим модель с заданной глубиной дерева
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)

    # обучим модель
    model.fit(features_train, target_train)
    
    # найдем предсказания на валидационной выборке 
    predictions_valid = model.predict(features_valid)
    


    print("Максимальная глубина деревьев = ", depth, ": ", end='')
    print("Accuracy наилучшей модели на валидационной выборке = ",accuracy_score(target_valid, predictions_valid))   

Максимальная глубина деревьев =  1 : Accuracy наилучшей модели на валидационной выборке =  0.7542768273716952
Максимальная глубина деревьев =  2 : Accuracy наилучшей модели на валидационной выборке =  0.7822706065318819
Максимальная глубина деревьев =  3 : Accuracy наилучшей модели на валидационной выборке =  0.7853810264385692
Максимальная глубина деревьев =  4 : Accuracy наилучшей модели на валидационной выборке =  0.7791601866251944
Максимальная глубина деревьев =  5 : Accuracy наилучшей модели на валидационной выборке =  0.7791601866251944
Максимальная глубина деревьев =  6 : Accuracy наилучшей модели на валидационной выборке =  0.7838258164852255
Максимальная глубина деревьев =  7 : Accuracy наилучшей модели на валидационной выборке =  0.7822706065318819
Максимальная глубина деревьев =  8 : Accuracy наилучшей модели на валидационной выборке =  0.7791601866251944
Максимальная глубина деревьев =  9 : Accuracy наилучшей модели на валидационной выборке =  0.7822706065318819


**Вывод:**

Судя по валидационной выборке, решающее дерево имеет самую высокую оценку правильности 0.7853810264385692, когда задан гиперпараметр глубины равный 3.

## Случайный лес

Попробуем алгоритм классификации — **случайный лес** (англ. random forest). Алгоритм обучает большое количество независимых друг от друга деревьев, а потом принимает решение на основе голосования. Случайный лес помогает улучшить результат предсказания и избежать переобучения.

In [10]:
best_model = None
best_result = 0
for est in range(1,101):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) # обучим модель с заданным количеством деревьев
    model.fit(features_train, target_train) # обучим модель на тренировочной выборке
    result = model.score(features_valid, target_valid) # посчитаем качество модели на валидационной выборке
    if result > best_result:
        best_model = model # сохраним наилучшую модель
        best_result = result #  сохраним наилучшее значение метрики accuracy на валидационных данных
        print("Число деревьев = ", est, ": ", end='')        
        print("Accuracy наилучшей модели на валидационной выборке =", best_result)

Число деревьев =  1 : Accuracy наилучшей модели на валидационной выборке = 0.7107309486780715
Число деревьев =  2 : Accuracy наилучшей модели на валидационной выборке = 0.7636080870917574
Число деревьев =  4 : Accuracy наилучшей модели на валидационной выборке = 0.7713841368584758
Число деревьев =  6 : Accuracy наилучшей модели на валидационной выборке = 0.7807153965785381
Число деревьев =  8 : Accuracy наилучшей модели на валидационной выборке = 0.7822706065318819
Число деревьев =  10 : Accuracy наилучшей модели на валидационной выборке = 0.7853810264385692
Число деревьев =  12 : Accuracy наилучшей модели на валидационной выборке = 0.7869362363919129
Число деревьев =  18 : Accuracy наилучшей модели на валидационной выборке = 0.7931570762052877
Число деревьев =  23 : Accuracy наилучшей модели на валидационной выборке = 0.7947122861586314


**Вывод:**

Модель случайного леса предсказывает тариф точнее, но, как мы видим, не на много – 0.7947122861586314, когда задано количество деревьев 23.

## Логистическая регрессия

Если сделать гиперпараметр n_estimators больше, модель начнёт разрастаться и медленно обучаться. Это плохо. Мало деревьев и результаты не лучше. Тоже плохо. Сколько можно быть зависимым от деревьев? Попробуем ещё один алгоритм — **логистическую регрессию**.

In [11]:
model = LogisticRegression(random_state=12345, solver='lbfgs',max_iter=1000) # инициализируем модель логистической регрессии с параметром random_state=12345
model.fit(features_train, target_train) # обучим модель на тренировочной выборке
result = model.score(features_train, target_train) # получим метрику качества модели на валидационной выборке
print(result)

0.7131742738589212


**Вывод:**

Логистическая регрессия дала самый худший результат – 0.7131742738589212.

**Вывод:**
    
    
- Лучше всех показал себя случаный лес -0.7947122861586314 c количеством деревьев 23
- На втором месте немного остает решающее дерево - 0.7853810264385692 c решающей глубиной 3
- Ото всех отстает логистическая регрессия с результатом – 0.7131742738589212


# Проверим качество модели на тестовой выборке

Одновременно работать с тремя моделями не нужно. У каждой — свои достоинства и недостатки. Оценим модели по качеству (accuracy) и скорости работы.

1. Качество (accuracy). Это самый важный критерий для бизнеса: чем выше качество, тем больше прибыли приносит продукт.
- Самое высокое качество у случайного леса: вместо одного решающего дерева используется целый ансамбль.
- На втором месте - дерево решений. Если глубина меньше трех, оно недообучается, когда больше — переобучается.
- Самое низкое качество предсказания у логистической регрессии.Модель несложная, а значит, переобучение ей не грозит.

2. Скорость работы. Не менее значимый критерий: если сервис работает медленно, оттока пользователей не избежать.
- Высокая скорость работы у логистической регрессии: у неё меньше всего параметров.
- Скорость решающего дерева тоже высокая и зависит от глубины. Помним, в экспериментах наилучшее качество модели получилось при глубине, равной 3. Ответ на вопрос модель получила всего-то за три проверки значений признаков — а это очень быстро!
- Случайный лес медленнее всех: чем больше деревьев, тем неторопливее работает модель.

Проверим как поведет себя модель на тестовой выборке

In [12]:
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

model = DecisionTreeClassifier(random_state=12345,max_depth=3)
model.fit(features_train, target_train)

predictions_train = model.predict(features_train)
predictions_test = model.predict(features_test)


accuracy_score(target_test, predictions_test)

print("Обучающая выборка:" ,accuracy_score(target_train, predictions_train)) 
print("Тестовая выборка:" ,accuracy_score(target_test, predictions_test)) 

Обучающая выборка: 0.8075726141078838
Тестовая выборка: 0.7791601866251944


**Вывод:**
На тестовой выборке модель повела приближенно к тренировочной выборке,что свидетельствует о том,что нет проблем с недообучением или переобучением   

In [13]:
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

model = RandomForestClassifier(random_state=12345, n_estimators=23) 
model.fit(features_train, target_train)

predictions_train = model.predict(features_train)
predictions_test = model.predict(features_test)


accuracy_score(target_test, predictions_test)

print("Обучающая выборка:" ,accuracy_score(target_train, predictions_train)) 
print("Тестовая выборка:" ,accuracy_score(target_test, predictions_test)) 

Обучающая выборка: 0.9937759336099585
Тестовая выборка: 0.7807153965785381


**Вывод:**

Accuracy на тестовой выборке модели ниже, чем на обучающей.
Модель хорошо объяснила примеры из обучающего набора данных, но запуталась в тестовой выборке и не смогла ответить правильно. Мы столкнулись с проблемой переобучения.

# Проверим модели на вменяемость

**DummyClassifier** делает прогнозы, которые игнорируют входные функции.
Этот классификатор служит простой базовой линией для сравнения с другими более сложными классификаторами

In [14]:
dummy_clf = DummyClassifier(strategy='most_frequent', random_state=0)
dummy_clf.fit(features_train, target_train)
dummy_clf.score(features_train, target_train)

0.6924273858921162

Здесь мы можем использовать DummyClassifier, чтобы всегда предсказать «не мошенничество», просто чтобы показать, насколько может быть обманчива точность.

In [15]:
dummy = DummyClassifier(strategy='most_frequent').fit(features_train, target_train)
dummy_pred = dummy.predict(features_test)
accuracy_score(target_test, dummy_pred)

0.6842923794712286

Мы получили оценку точности 68,4 %

**Вывод:**

Таким образом, мы можем считать модель адекватной для использования.

# Общий вывод

- 2 из 3 моделей показали значение правильных ответов более 0.75 - это решающее дерево с Accuracy 0.78 и случайный лес с Accuracy 0.79, самый худший показател у логистической регрессии и 0.71. 
- Лучше всех показал себя алгоритм классификации — **решающее дерево**.
- Модель можно считать вменяемой